In [3]:
'''
使用colab直接運行下載
'''
import sys
import torch
pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{pyt_version_str}"
])
!pip install fvcore iopath
!pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 904.1 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=a4beaec33e6979568df85af04cf83b1ae3e0100e9b8718011be718d48c68b82a
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31530 sha256=585f1eabc33326be93b709d0ba059f1b517154438680bcb087beead265b69c89
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath
Looking in links: https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/py310_cu118_pyt201/download.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 154.0 MB/s eta 0:00:00


In [5]:
import pytorch3d

In [6]:
import torch
import numpy as np
import math
from pytorch3d.renderer import (FoVPerspectiveCameras,PointLights,look_at_view_transform,NDCGridRaysampler)

In [7]:
if torch.cuda.is_available():
  device = torch.device('cuda:0')
  torch.cuda.set_device(device)
else:
  device = torch.device('cpu')

定義一個批次有10個相機 = num_views
elev = elev 變數表示仰角，azim 表示方位角。
旋轉角R
translation = T

In [8]:
num_views: int = 10
azimuth_range: float = 180
elev = torch.linspace(0,0,num_views)
azim = torch.linspace(-azimuth_range,azimuth_range,num_views)+180.0
lights = PointLights(device=device,location=[[0.0,0.0,-3.0]])

In [9]:
R,T = look_at_view_transform(dist=2.7,elev=elev,azim=azim)
cameras = FoVPerspectiveCameras(device=device,R=R,T=T)

定義 ray sampler

In [10]:
image_size = 64
volume_extent_world = 3.0
raysampler = NDCGridRaysampler(
    image_width = image_size,
    image_height=image_size,
    n_pts_per_ray=50,
    min_depth = 0.1,
    max_depth = volume_extent_world,
)


在上一步中，我們已經定義了光線採樣器。 為了讓光線採樣器採樣光線和點以供使用，我們需要讓光線採樣器知道我們的相機在哪裡以及它們指向什麼方向。 透過將步驟 3 中定義的相機傳遞給光線採樣器，可以輕鬆實現這一點。 那我們得到的是一個 ray_bundle 變數：

In [11]:
ray_bundle = raysampler(cameras)

In [12]:
print('ray_bundle origins tensor shape = ', ray_bundle.
origins.shape)
print('ray_bundle directions shape = ', ray_bundle.
directions.shape)
print('ray_bundle lengths = ', ray_bundle.lengths.shape)
print('ray_bundle xys shape = ',ray_bundle.xys.shape)

ray_bundle origins tensor shape =  torch.Size([10, 64, 64, 3])
ray_bundle directions shape =  torch.Size([10, 64, 64, 3])
ray_bundle lengths =  torch.Size([10, 64, 64, 50])
ray_bundle xys shape =  torch.Size([10, 64, 64, 2])


In [ ]:
torch.save({
    'ray_bundle':ray_bundle,
},'ray_sampling.pt')